In [14]:
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset
from torch import nn, optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchsummary import summary
import seaborn as sns
from copy import deepcopy

# Req for package
import sys
sys.path.append("../")
from SkinLearning.NN.Helpers import train, test, DEVICE, getParameterLoss, setSeed
from SkinLearning.NN.Models import MultiTemporal
from SkinLearning.Utils.Dataset import getDataset, getSplit
from SkinLearning.Utils.Plotting import plotParameterBars


torch.backends.cudnn.benchmark = True

In [2]:
setSeed()

In [3]:
dataset, scaler = getDataset()

100%|█████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:09<00:00, 226.18it/s]


In [4]:
train_loader, test_loader = getSplit(dataset)

In [19]:
lstm1 = MultiTemporal(out="f_output", single_fc=False, temporal_type="LSTM")
#lstm2 = MultiTemporal(out="f_output", single_fc=False, temporal_type="LSTM", old=False)

In [16]:
def test_k_fold(dataset, model, scaler,k=5, ):
    # Initialize k and KFold
    kfold = KFold(n_splits=k, shuffle=True)

    # Perform k-fold cross-validation
    accuracies = []

    for fold, (train_index, valid_index) in enumerate(kfold.split(dataset), start=1):
        print(f"Testing fold {fold}")
        
        train_set = Subset(dataset, train_index)
        valid_set = Subset(dataset, valid_index)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        valid_loader = DataLoader(valid_set, batch_size=32)

        # Train the model
        train(train_loader, model, val_loader=valid_loader, LR=0.0001, epochs=300)

        accuracy, _, _ = test(valid_loader, model,scaler)
        accuracies.append(accuracy)
        print(f"Fold {fold} accuracy: {accuracy:.2f}")

    # Calculate average accuracy across all folds
    average_accuracy = np.mean(accuracies)
    print(f"Average accuracy: {average_accuracy:.2f}")


In [17]:
import gc
torch.cuda.empty_cache()
gc.collect()

3368

In [ ]:
print(lstm)

In [ ]:
test_k_fold(dataset, lstm2, scaler) # 79

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
test(test_loader, lstm2, scaler, cluster=True)

In [22]:
test(test_loader, lstm1, scaler)

(46.195518493652344,
 array([ 91.46532 ,  44.912945,  99.646706,  99.12827 , -89.05632 ,
         31.076202], dtype=float32),
 0.1683460146188736)

In [21]:
train(train_loader, lstm1, val_loader=test_loader, LR=0.0001, epochs=1500, early_stopping=True)

Using: cuda


 16%|████▌                       | 9/56 [00:00<00:01, 39.47batch/s, counter=0, epoch=12, lastLoss=0.165, valLoss=0.164]


KeyboardInterrupt: 

In [ ]:
train(train_loader, lstm2, early_stopping=True, epochs=1500, val_loader=test_loader)

In [ ]:
class MultiTemporal(nn.Module):
    def __init__(self, hidden_size=256, single_fc=True, out="f_hidden", layers=1, temporal_type="RNN", old=True):
        super(MultiTemporal, self).__init__()
        self.hidden_size = hidden_size
        self.out = out
        
        self.cnn = deepcopy(best_CNN)

        if temporal_type == "RNN":
            net = nn.RNN
        elif temporal_type == "LSTM":
            net = nn.LSTM
        elif temporal_type == "GRU":
            net = nn.GRU
        else:
            raise Exception("Not a valid NN type.")
            
        if not old:
            net_inp = 15
            self.cnn = deepcopy(new_CNN)
        else:
            net_inp = 6
            self.cnn = deepcopy(best_CNN)
        
        self.net = net(
            net_inp,
            hidden_size,
            batch_first=True,
            num_layers=layers,
            )
        
        # Check size of output to determine FC input
        input_tensor = torch.zeros(32, 512, net_inp)
        output, _ = self.net(input_tensor)
        
        fc_in = hidden_size
        if out == 'output':
            fc_in = output.shape[1] * output.shape[2]
        elif out == 'f-output':
            fc_in = output.shape[2]
        elif fc_in == 'hidden' or out == 'cell':
            out = hidden_size * output.shape[2]
        elif out == 'f-hiden' or out == 'f-cell':
            fc_in = output.shape[2]
        elif out == 'h+o' or out == 'h+c' :
            fc_in = output.shape[1]
        
        if single_fc:
            self.fc = nn.Linear(fc_in*layers, 6)
        else:
            self.fc = nn.Sequential(
                nn.Linear(fc_in*layers, 128),
                nn.ReLU(),
                nn.Linear(128 , 64),
                nn.ReLU(),
                nn.Linear(64, 6),   
            )

    def forward(self, x):
        batch_size = x.shape[0]
        x = self.cnn(x)
    
        o, (h, c) = self.net(x)
        
        if self.out == "f_hidden":
            x = h[-1].reshape(batch_size, -1)
        elif self.out == "hidden":
            x = h.reshape(batch_size, -1)
        elif self.out == "f_output":
            x = o[:, -1, :].reshape(batch_size, -1)
        elif self.out == "output":
            x = o.reshape(batch_size, -1)
        elif self.out == "h+c":
            x = torch.concat([h[-1], c[-1]], dim=1).view(batch_size, -1)
        elif self.out == "h+o":
            x = torch.concat([h[-1], o[:, -1, :]], dim=1).view(o.size(0), -1)
            
        x = self.fc(x)
        return x

In [ ]:
best_CNN = nn.Sequential(
    nn.Conv1d(2, 128, kernel_size=5, padding=1, bias=False),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.MaxPool1d(kernel_size=5, stride=5),

    nn.Conv1d(128, 256, kernel_size=3, padding=1, bias=False),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.MaxPool1d(kernel_size=2, stride=2),

    nn.Conv1d(256, 512, kernel_size=3, padding=1, bias=False),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.MaxPool1d(kernel_size=2, stride=2),
)

In [ ]:
new_CNN = nn.Sequential(
    nn.Conv1d(2, 128, kernel_size=5, padding=1, bias=False),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.MaxPool1d(kernel_size=5, stride=2),

    nn.Conv1d(128, 256, kernel_size=3, padding=1, bias=False),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.MaxPool1d(kernel_size=2, stride=2),

    nn.Conv1d(256, 512, kernel_size=3, padding=1, bias=False),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.MaxPool1d(kernel_size=2, stride=2),
)

In [10]:
from SkinLearning.NN.Models import DualDown, DualDownUp, DualUp, DualUpDown
from SkinLearning.NN.Helpers import KFCV
models = [
       # nn.DataParallel(DualDownUp()),
        nn.DataParallel(DualUp()),
       # nn.DataParallel(DualDown()),
       # nn.DataParallel(DualUpDown())

    ]
    
names = [
    #"Down Sample/Up Sample",
    "Up Sample",
    #"Down Sample",
    #"Up Sample/Down Sample"
]

#dataset, scaler = getDataset()

accuracies = []
p_accs = []
for i, model in enumerate(models):
    print("--------------------------------------------------")
    print(f"Running model: {names[i]}) ({i+1}/{len(names)}")
    print("--------------------------------------------------")

    acc, p_acc = KFCV(dataset, model, scaler, cluster=False)
    accuracies.append(acc)
    p_accs.append(p_acc)

--------------------------------------------------
Running model: Up Sample) (1/1
--------------------------------------------------
Testing fold 1
Using: cuda


  0%|                                                                                        | 0/56 [00:00<?, ?batch/s]


RuntimeError: Given input size: (1024x1x1). Calculated output size: (1024x1x0). Output size is too small